# 📘 Tutoriel Détaillé UNLOCBOX : La Puissance de l'Optimisation Convexe Proximal

Ce Notebook interactif MATLAB (Live Script) est conçu pour les utilisateurs souhaitant appliquer l'**Optimisation Convexe** à des problèmes d'ingénierie ou d'analyse de données, en utilisant la boîte à outils **UNLocBoX**. Nous allons explorer le concept clé de l'**Opérateur Proximal** et l'appliquer au problème fondamental de la **Régression *Fused Lasso***.

## 1\. Concepts Fondamentaux de l'Optimisation Convexe

### 1.1. Pourquoi l'Optimisation Convexe ? 

L'optimisation convexe est une branche de l'optimisation mathématique où l'on minimise une fonction objectif convexe sur un ensemble de contraintes convexe. La propriété cruciale est que **tout minimum local est un minimum global**, garantissant que nos algorithmes trouvent la meilleure solution possible (pour peu qu'une solution existe).

### 1.2. Le Principe du *Proximal Splitting* 

Lorsque la fonction objectif $f(x)$ peut être décomposée en une somme de termes $f(x) = f_1(x) + f_2(x) + \dots + f_K(x)$, les méthodes classiques de gradient peuvent être inefficaces ou impossibles à appliquer si certains termes $f_n$ sont **non-différentiables** (par exemple, la norme $l_1$, souvent utilisée pour l'induction de parcimonie).

Le *proximal splitting* contourne cela en traitant chaque terme séparément.

| Type de Fonction | Caractéristique | Outil Requis pour UNLocBoX |
| :--- | :--- | :--- |
| **Lisse** (*Smooth*) | Différentiable | **Gradient** $\nabla f_n(x)$ et **Constante de Lipschitz** $\beta$ |
| **Non-Lisse** (*Nonsmooth*) | Non-différentiable | **Opérateur Proximal** $prox_{\lambda f_n}(x)$ |



## 2\. Installation et Initialisation d'UNLocBoX

### 2.1. Préparation de l'Environnement 

Commencez par ajouter le chemin d'UNLocBoX à votre session MATLAB et exécutez le script d'initialisation.

In [1]:

% Assurez-vous d'avoir téléchargé UNLocBoX (souvent nommé unlocbox-master)
% Naviguez dans le répertoire racine d'UNLocBoX.

% 1. Ajout du chemin (remplacez par votre chemin réel)
addpath(genpath('./unlocbox-master')); 

% 2. Initialisation de la boîte à outils
init_unlocbox;

disp(' UNLocBoX a été initialisé. Nous sommes prêts à résoudre des problèmes d''optimisation.');


UnLocBoX version 1.8.0. Copyright 2012-2015 LTS2-EPFL, by Nathanael Perraudin
 UNLocBoX a été initialisé. Nous sommes prêts à résoudre des problèmes d'optimisation.


## 3\. Le Cas d'Étude : La Régression Fused Lasso

Nous allons résoudre le problème de la **Régression *Fused Lasso***, qui est idéal pour illustrer le *proximal splitting* car il comporte trois termes :

$$\min_{x} \underbrace{\frac{1}{2}||Ax-y||_{2}^{2}}_{f_{1}(x)} + \underbrace{\lambda_1 ||x||_{1}}_{f_{2}(x)} + \underbrace{\lambda_2 ||Dx||_{1}}_{f_{3}(x)}$$

Où :

  * $f_1(x)$ est le terme d'ajustement aux données (lisse).
  * $f_2(x)$ est la régularisation **Lasso** (induit la parcimonie, non-lisse).
  * $f_3(x)$ est la régularisation ***Fused*** (induit la parcimonie des différences, non-lisse).
  * $D$ est la matrice de différence finie, et $\lambda_1, \lambda_2$ sont les coefficients de pondération.

### 3.1. Simulation des Données 

Générons des données synthétiques pour notre problème de régression.

In [2]:
% 1. Définition des dimensions et des paramètres
N = 100; % Longueur du signal (nombre de variables)
M = 50;  % Nombre d'observations
lambda1 = 0.5; % Poids du terme Lasso
lambda2 = 1.5; % Poids du terme Fused Lasso

% 2. Création de la matrice de mesure (A)
A = randn(M, N);
A = A ./ norm(A); % Normalisation

% 3. Création du signal vrai (sparse et parcimonieux en différences)
x_true = zeros(N, 1);
x_true(20:40) = 5;
x_true(60:80) = -3; 
y = A * x_true + 0.1 * randn(M, 1); % Observation bruitée

% 4. Matrice de différence finie (D)
D = spdiags([-ones(N, 1) ones(N, 1)], [0 1], N-1, N); % D*x approxime la dérivée

% 5. Point d'initialisation
x0 = zeros(N, 1); 

disp(['Taille du problème: N = ', num2str(N), ', M = ', num2str(M)]);

Taille du problème: N = 100, M = 50



### 3.2. Définition de la Fonction Lisse $f_1(x)$ 

La fonction $f_1(x) = \frac{1}{2}||Ax-y||_{2}^{2}$ est quadratique.

#### 3.2.1. Calcul de la Constante de Lipschitz ($\beta$) 

Pour les fonctions lisses, l'algorithme *Forward-Backward* (FISTA) nécessite la constante de Lipschitz du gradient.
$$\beta = \text{Lipschitz}(\nabla f_1) = ||\nabla^2 f_1||_2 = ||A^T A||_2 = ||A||_2^2$$

In [3]:

% Structure pour la fonction Lisse f1
f1.eval = @(x) 0.5 * norm(A*x - y)^2;
f1.grad = @(x) A' * (A*x - y);
f1.beta = norm(A)^2; % Calcul de la constante de Lipschitz
f1.name = 'Data Fidelity (L2 squared)';

disp('Détails f1: eval, grad, beta définis.');

Détails f1: eval, grad, beta définis.



### 3.3. Définition de la Régularisation Lasso $f_2(x)$ 

$f_2(x) = \lambda_1 ||x||_{1}$ est non-lisse. Nous devons utiliser l'opérateur proximal `prox_l1`.

#### 3.3.1. L'Opérateur Proximal $\text{prox}_{\tau \lambda_1 ||\cdot||_1}$ 

L'opérateur proximal de $\lambda_1 ||\cdot||_1$ est la fonction de **seuil souple** (*Soft Thresholding*), essentielle pour l'induction de parcimonie.


In [4]:

% Structure pour la fonction Non-Lisse f2 (Lasso)
f2.eval = @(x) lambda1 * norm(x, 1);
% L'opérateur prox_l1 prend x, et le pas de temps T (tau) qui est multiplié par lambda1
f2.prox = @(x, T) prox_l1(x, T * lambda1); 
f2.name = 'Lasso Regularization (L1)';

disp('Détails f2: eval, prox (Soft Thresholding) définis.');

Détails f2: eval, prox (Soft Thresholding) définis.


### 3.4. Définition de la Régularisation Fused Lasso $f_3(x)$ 

$f_3(x) = \lambda_2 ||Dx||_{1}$ est aussi non-lisse. UNLocBoX gère cela avec l'opérateur proximal `prox_l1` appliqué dans l'espace des différences.


In [5]:
% Structure pour la fonction Non-Lisse f3 (Fused Lasso)
f3.eval = @(x) lambda2 * norm(D*x, 1);

% Pour appliquer prox_l1 à D*x, nous devons utiliser un opérateur de projection
% prox_l1_linop, qui est un opérateur proximal avancé pour les fonctions de la forme f(L*x).
% Nous devons définir l'opérateur linéaire (L=D) et son adjoint (D')
param_fl.A = D;     % L'opérateur linéaire
param_fl.At = D';   % Son adjoint (transposée)

% Le prox_l1_linop utilise le pas de temps T pour l'algorithme ADMM ou Douglas-Rachford.
f3.prox = @(x, T) prox_l1_linop(x, T * lambda2, param_fl); 
f3.name = 'Fused Lasso Regularization (L1 on differences)';

disp('Détails f3: eval, prox (l1 sur opérateur linéaire) définis.');

Détails f3: eval, prox (l1 sur opérateur linéaire) définis.


## 4\. Choix du Solveur et Résolution

Le problème a la forme $f_1(x) + f_2(x) + f_3(x)$, soit une fonction lisse ($f_1$) plus deux fonctions non-lisses ($f_2$ et $f_3$).

  * **Solveurs adaptés :**
      * **Douglas-Rachford (DR) :** Excellent pour les sommes de deux fonctions non-lisses. Nécessite de grouper $f_1+f_2$ (lisse + non-lisse) ou $f_2+f_3$ (non-lisse + non-lisse).
      * **Forward-Backward (FISTA) :** Idéal pour $f_{\text{lisse}} + f_{\text{non-lisse}}$.

Puisque nous avons trois termes, nous utiliserons la fonction générique `solvep` qui choisira le meilleur algorithme (souvent une version du **Douglas-Rachford** ou du **SDMM** qui gère la somme de trois prox).

### 4.1. Paramétrage Général 

Définissons les paramètres pour tous les solveurs.

In [6]:
param.verbose = 2; % Afficher les informations détaillées
param.maxit = 2000; % Augmentation des itérations pour la convergence
param.tol = 1e-4; % Tolérance d'arrêt
param.method = 'solvep'; % Utilisation du solveur universel

% 💡 Plugin de monitoring (Optionnel : pour afficher les résultats en cours de route)
fig = figure(1);
param.do_sol = @(x) plot(x_true, 'b', x, 'r--', 'LineWidth', 1.5);
param.rel_obj = 1; % Calculer la différence relative entre les itérations

disp('Paramètres du solveur définis. Résolution en cours...');

Paramètres du solveur définis. Résolution en cours...



### 4.2. Exécution et Résultats 

Lançons l'optimisation en passant l'ensemble des structures au solveur `solvep`.

In [7]:
% Résolution du problème Fused Lasso complet
sol_fl = solvep(x0, {f1, f2, f3}, param); 

% Affichage final
figure(2);
plot(x_true, 'b', 'LineWidth', 2);
hold on;
plot(sol_fl, 'r--', 'LineWidth', 1.5);
title('Résolution Fused Lasso par UNLocBoX');
legend('Signal Vrai (x\_true)', 'Solution Fused Lasso (sol\_fl)');
xlabel('Indice du Signal');
ylabel('Valeur');
grid on;

disp('✅ Optimisation terminée. La solution est affichée.');

A function has both the prox and a grad fields. The gradient is used
Algorithm selected: FB_BASED_PRIMAL_DUAL 
Iter 001:     prox_L1: ||A x-y||_1 = 1.021061e+01, REL_OB, iter = 2


Error using fb_based_primal_dual_alg>fb_based_primal_dual_algorithm (line 158)
Unknown method

Error in fb_based_primal_dual_alg>@(x_0,fg,Fp,sol,s,param)fb_based_primal_dual_algorithm(fg,Fp,sol,s,param) (line 21)
    fb_based_primal_dual_algorithm(fg, Fp, sol, s, param);

Error in solvep (line 209)
    [sol, s] = algo.algorithm(x_0, fg, Fp, sol, s, param);


## 5\. Algorithmes Spécifiques et Options Avancées

UNLocBoX vous permet d'appeler directement des algorithmes pour un contrôle plus fin.

### 5.1. Algorithme FISTA (*Fast Iterative Shrinkage-Thresholding Algorithm*) 

FISTA est l'implémentation accélérée du *Forward-Backward*. Il ne peut résoudre que $f_{\text{lisse}} + f_{\text{non-lisse}}$.

Pour l'utiliser, nous devons grouper $f_2$ et $f_3$ dans un terme non-lisse unique $g(x) = f_2(x) + f_3(x)$, puis résoudre $\min f_1(x) + g(x)$ avec `forward_backward`. Cependant, trouver le `prox` de $g(x)$ est un autre problème d'optimisation complexe (sauf cas spéciaux), ce qui justifie l'utilisation de **DR** ou **Chambolle-Pock** pour trois termes.

### 5.2. Algorithme Douglas-Rachford (DR) 

DR est efficace pour résoudre $\min f_{A}(x) + f_{B}(x)$, où $f_A$ et $f_B$ sont toutes deux non-lisses (ou lisses). Nous pouvons le forcer à résoudre $f_1(x) + (f_2(x)+f_3(x))$ en regroupant deux termes.


In [11]:
% Pour DR, nous allons résoudre : min (f1 + f2) + f3
% Cette décomposition nécessite la somme des prox de f1 et f2.
% Dans ce cas, il est plus simple de laisser solvep gérer les trois termes, 
% ou d'utiliser une fonction comme 'sum_prox' si elle est disponible.

% Alternative : Utiliser DR pour résoudre min f1 + f2 (sans f3 pour l'exemple)
% f1 est lisse, mais DR peut quand même la gérer en tant que prox.
% param_dr = param;
% param_dr.maxit = 500;
% sol_dr = douglas_rachford(x0, f1, f2, param_dr);
% disp('Résolu avec Douglas-Rachford (pour f1 + f2).');

## 6\. Conclusion et Ressources 

Vous avez appris à :

1.  Structurer des fonctions convexes (lisses ou non-lisses) pour UNLocBoX.
2.  Utiliser les concepts clés de **gradient**, de **constante de Lipschitz** et d'**opérateur proximal**.
3.  Résoudre un problème d'optimisation complexe (*Fused Lasso*) à trois termes à l'aide de la fonction générique `solvep`.

UNLocBoX offre une flexibilité incroyable, permettant d'assembler des briques de fonctions comme des Legos pour créer des problèmes d'optimisation sur mesure. Référez-vous au guide utilisateur complet pour la liste des opérateurs proximaux disponibles (`prox_tv`, `prox_l2`, `prox_positive`, etc.) et des solveurs avancés (Chambolle-Pock, SDMM).